In [1]:
import requests
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ast
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, to_tree
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
import kneed
from collections import deque

import sys, os

sys.setrecursionlimit(2000)

In [2]:
def read_labels(path, dataset='ss-role'):
    label_data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            if dataset in ['ss-func', 'ss-role']:
                word_info, word_label = line.strip().split('\t')
                sent_info, word = word_info.split(':')
                sent_info = ast.literal_eval(sent_info)
                label_data.append([sent_info[0], sent_info[1], word, word_label])
            elif dataset == 'dep':
                word_pair, word_label = line.strip().split('\t')
                word_pair = word_pair.split('--')
                if len(word_pair) == 2:
                    word1, word2 = word_pair
                elif len(word_pair) == 3:
                    word1, word2 = word_pair[0], '--'
                sent_info = [0, 0]
                label_data.append([sent_info[0], sent_info[1], f'{word1}--{word2}', word_label])
            else:
                raise ValueError('Dataset not supported')

    return pd.DataFrame(label_data, columns=['sent_id', 'word_id', 'word', 'label'])


def get_graph(dataset, dist_metric, filter, intervals, overlap, iteration, layer, datasplit):
    # make request to local server at port 5000 at \graph
    # with the query: {params: 'ss-role_euclidean_l2_50_50', iteration: 0, layer: 12, datasplit: 'train'}
    # and save the response as a variable
    r = requests.get(
        'http://localhost:5000/graph',
        params={'params': f'{dataset}_{dist_metric}_{filter}_{intervals}_{overlap}', 'iteration': iteration, 'layer': layer,
                'datasplit': datasplit})
    data = r.json()
    graph = nx.json_graph.node_link_graph(data['graph'])
    return graph


def get_activations(dataset, iteration, layer, datasplit):
    activations = pd.read_csv(
        f'../backend/data/{dataset}/fine-tuned-bert-base-uncased/{datasplit}/{iteration}/{layer}.txt', delim_whitespace=True, header=None)
    labels = read_labels(f'../backend/data/{dataset}/entities/{datasplit}.txt', dataset=dataset)

    return activations, labels


def point_to_node(nodes, num_points=4282):
    ptn_dict = {}
    for point_idx in range(num_points):
        for node_idx, node in enumerate(nodes):
            node_data = nodes[node]['membership']['membership_ids']

            if point_idx in node_data:
                ptn_dict[point_idx] = node_idx

    return ptn_dict


def linkage_matrix(model):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_, counts]).astype(float)

    return linkage_matrix


def plot_dendrogram(model):
    linkage_mat = linkage_matrix(model)

    # Plot the corresponding dendrogram
    return dendrogram(linkage_mat, orientation='left', truncate_mode='level', p=8, leaf_font_size=12, leaf_label_func=leaf_label)


def leaf_label(idx):
    if idx < len(labels):
        return labels.iloc[idx]['label']
    else:
        return 'None'


def elbow_eps(data):
    nbrs = NearestNeighbors(n_neighbors=2).fit(data)
    distances, indices = nbrs.kneighbors(data)
    distances = np.sort(distances, axis=0)[::-1]
    kneedle = kneed.KneeLocator(distances[:, 1], np.linspace(0, 1, num=len(distances)), curve='convex', direction='decreasing')
    eps = kneedle.knee * 0.75
    return eps


def populate_tree_labels(treeNode, labels):
    if treeNode.id < len(labels):
        treeNode.label = set([labels.iloc[treeNode.id]['label'].split('.')[1]])
    else:
        # popluate children
        populate_tree_labels(treeNode.left, labels)
        populate_tree_labels(treeNode.right, labels)

        # label is union of labels of left and right children
        treeNode.label = treeNode.left.label | treeNode.right.label

    return treeNode


def process_label(label, max_length=15):
    label_str = ','.join(sorted(label))
    if len(label_str) > max_length:
        return f'{label_str[:max_length]}... ({len(label)})'
    else:
        return label_str


def bfs_traversal(treeNode, graph, max_level=5):
    q = deque()

    q.append(treeNode)
    level = 0

    while len(q) > 0 and level < max_level:
        level_size = len(q)

        for _ in range(level_size):
            node = q.popleft()
            num_labels = len(node.label)

            graph.add_node(node.id, label=process_label(node.label))

            # terminate if node's label has one label
            if num_labels > 1:
                if node.left is not None:
                    q.append(node.left)
                    graph.add_edge(node.id, node.left.id, weight=node.left.dist)
                    graph.add_node(node.left.id, label=process_label(node.left.label))


                if node.right is not None:
                    q.append(node.right)
                    graph.add_edge(node.id, node.right.id, weight=node.right.dist)
                    graph.add_node(node.right.id, label=process_label(node.right.label))

        level += 1


def node_to_point_matrix(activations, ptn_dict, node_dist_matrix):
    point_dist_mat_from_graph = np.zeros((len(activations), len(activations)))

    # populate point_dist_mat_from_graph
    for point_idx1 in range(len(activations)):
        for point_idx2 in range(len(activations)):
            if point_idx1 not in ptn_dict or point_idx2 not in ptn_dict:
                point_dist_mat_from_graph[point_idx1][point_idx2] = 100
            elif point_idx1 != point_idx2:
                point_dist_mat_from_graph[point_idx1, point_idx2] = node_dist_matrix[ptn_dict[point_idx1], ptn_dict[point_idx2]]

    return point_dist_mat_from_graph


In [3]:
def induced_hierarchy(dataset, iteration, layer, datasplit, max_level=10, dist_metric='euclidean', filter='l2', intervals=50, overlap=50):
    filename = f'images/hierarchies/{dataset}_{datasplit}_iter{iteration}_layer{layer}_level{max_level}.svg'

    # if file exists, return
    if os.path.isfile(filename):
        print('File exists, returning')
        return
        
    # get mapper graph
    graph = get_graph(dataset, dist_metric, filter=filter, intervals=intervals,
                      overlap=overlap, iteration=iteration, layer=layer, datasplit=datasplit)
    # graph = get_graph(DATASET, 'euclidean', filter='l2', intervals=50, overlap=50, iteration=ITERATION, layer=LAYER, datasplit='train')

    # get activations and labels
    activations, labels = get_activations(dataset, iteration, layer, datasplit)

    # shortest path distance using distance between node centroids as the metric
    distance_matrix = nx.algorithms.shortest_paths.dense.floyd_warshall_numpy(graph, weight='centroid_dist')
    
    # set disconnected node distance to max + euclidean distance
    max_distance = np.ma.masked_invalid(distance_matrix).max()
    nodelist = list(graph.nodes())

    for i in range(len(distance_matrix)):
        for j in range(len(distance_matrix)):
            if np.isinf(distance_matrix[i][j]):
                centroid_i = np.array(graph.nodes[nodelist[i]]['membership']['centroid'])
                centroid_j = np.array(graph.nodes[nodelist[j]]['membership']['centroid'])
                distance_matrix[i][j] = max_distance + np.linalg.norm(centroid_i - centroid_j)

    # return distance_matrix

    # convert node-to-node distance matrix to point-to-node distance matrix
    ptn_dict = point_to_node(graph.nodes)
    point_dist_mat_from_graph = node_to_point_matrix(activations, ptn_dict, distance_matrix)

    # Perform hierarchical clustering using pointwise distance matrix
    model_aggclust_mapper = AgglomerativeClustering(n_clusters=None, distance_threshold=99, affinity='precomputed', linkage='average')
    model_aggclust_mapper.fit(point_dist_mat_from_graph)

    # compute the linkage matrix
    linkage_matrix_mapper = linkage_matrix(model_aggclust_mapper)
    tree = to_tree(linkage_matrix_mapper)
    populate_tree_labels(tree, labels)

    # Create a networkx directional graph
    linkage_graph = nx.DiGraph()

    # Populate the graph with using BFS traversal of the tree
    bfs_traversal(tree, linkage_graph, max_level=max_level)

    # Save SVG file of the graph
    Ag = nx.nx_agraph.to_agraph(linkage_graph)
    Ag.layout(prog='neato')
    Ag.draw(filename, format='svg', args="-Nshape=box")

    return linkage_graph


# # Train Layer 12
# induced_hierarchy('ss-role', iteration=175, layer=12, datasplit='train', max_level=50)
# induced_hierarchy('ss-role', iteration=5, layer=12, datasplit='train', max_level=50)

# # Train Layer 0
# induced_hierarchy('ss-role', iteration=175, layer=0, datasplit='train', max_level=50)
# induced_hierarchy('ss-role', iteration=5, layer=0, datasplit='train', max_level=50)

# # Test Layer 12
# induced_hierarchy('ss-role', iteration=175, layer=12, datasplit='test', max_level=50)
# induced_hierarchy('ss-role', iteration=5, layer=12, datasplit='test', max_level=50)

# # Test Layer 0
# induced_hierarchy('ss-role', iteration=175, layer=0, datasplit='test', max_level=50)
# induced_hierarchy('ss-role', iteration=5, layer=0, datasplit='test', max_level=50)


In [ ]:
iteration_list = [5, 65, 175]
layer_list = [0, 6, 9, 12]
datasplit_list = ['train', 'test']

for iteration in iteration_list:
    for layer in layer_list:
        for datasplit in datasplit_list:
            induced_hierarchy('ss-role', iteration=iteration, layer=layer, datasplit=datasplit, max_level=50)


In [74]:
# model_aggclust_mapper = AgglomerativeClustering(n_clusters=None, distance_threshold=50, affinity='precomputed', linkage='average')
# model_aggclust_mapper.fit(point_dist_mat_from_graph)

# model_aggclust_act = AgglomerativeClustering(n_clusters=None, distance_threshold=50, linkage='average')
# model_aggclust_act.fit(activations)

# model_dbscan_act = DBSCAN(eps=elbow_eps(activations), min_samples=1)
# model_dbscan_act.fit(activations)

AgglomerativeClustering(distance_threshold=50, linkage='average',
                        n_clusters=None)

In [96]:
def plot_data(data, color):
    xy = pd.DataFrame(PCA(n_components=2).fit_transform(activations), columns=['x', 'y'])

    # plot scatterplot using seaborn
    sns.scatterplot(data=xy, x='x', y='y', hue=color, s=100, alpha=0.5)

    plt.xlabel('x')
    plt.ylabel('y')

plt.figure(figsize=(25, 20))
plot_data(activations, pd.Series(model_dbscan_act.labels_).astype(str))

NameError: name 'model_dbscan_act' is not defined

<Figure size 1800x1440 with 0 Axes>

# Meeting summary
- Ran hierarchy extraction algorithm and visualization 
  - Using a modified version of the shortest path distance in the mapper graph compute the distance matrix for the points
  - [TODO] Replace max_distance with diameter (in euclidean sense) of activation space
    - [TODO] First check if diameter is larger than max tree distance, otherwise 
- [TODO] Show class distributions in the leaves instead of just distinct class names
- [TODO] Also add counts of labels to the nodes, and sort by majority classes
- [TODO] Show purity trends in induced hierarchy in epoch 0 vs epoch 175
  - x-axis: distance from root node, y-axis: purity distribution
- Human hierarchy and neural network derived topological hierarchy places where they align and misalign
  - Machine is unaware of human curated hierarchy - aligments make sense, but mismatches are interesting too
    - Reason for mismatch: shared words between categories? Improper formulation of metric? 
- Attaching the hierarchy SVGs for train data through layer 12, epoch 175 

Cross-task activations
- No major deviation from our hypothesis: Role And Func activation behave similarly, POS ones do not have much label purity


In [ ]:
# find maximim pairwise distance of activations
